In [1]:
import json
import pandas as pd

In [2]:
ber_path= "Rashi plaintext/json undisputed/Rashi on Berakhot - he - Vilna Edition.json"

In [3]:
# Load the JSON data
with open(ber_path, 'r', encoding='utf-8') as file:
    rashi_data = json.load(file)

    # Extract the Tractate name (assuming filename without extension for this example)
tractate_name = rashi_data['title'].split()[-1]  # Modify as needed

In [4]:
tractate_name

'Berakhot'

In [5]:
# Extract comments with detailed references (page, line, comment number)
comments_with_detailed_references = [(comment, daf_num, line_num, comment_num)
                                     for daf_num, daf in enumerate(rashi_data["text"])
                                     for line_num, line in enumerate(daf)
                                     for comment_num, comment in enumerate(line)]

In [6]:
comments_with_detailed_references[0]

('מאימתי קורין את שמע בערבין. משעה שהכהנים נכנסים לאכול בתרומתן – כהנים שנטמאו וטבלו והעריב שמשן והגיע עתם לאכול בתרומה:',
 2,
 0,
 0)

In [7]:
# Process the data and store in lists
tractate_data = []
location_data = []
verse_data = []
comment_data = []

for comment, daf_num, line_num, comment_num in comments_with_detailed_references:
    tractate_data.append(tractate_name)
    location_data.append((daf_num, line_num, comment_num))
    verse_data.append(comment.split("–")[0] if "–" in comment else "")
    comment_data.append(comment)

# Create a DataFrame
df = pd.DataFrame({
    'Tractate': tractate_data,
    'Location': location_data,
    'Starting Verse': verse_data,
    'Comment': comment_data
})

df.head()

,Tractate,Location,Starting Verse,Comment
0,Berakhot,"(2, 0, 0)",מאימתי קורין את שמע בערבין. משעה שהכהנים נכנסי...,מאימתי קורין את שמע בערבין. משעה שהכהנים נכנסי...
1,Berakhot,"(2, 0, 1)",עד סוף האשמורה הראשונה,עד סוף האשמורה הראשונה – שליש הלילה כדמפרש בגמ...
2,Berakhot,"(2, 2, 0)",עד שיעלה עמוד השחר,עד שיעלה עמוד השחר – שכל הלילה קרוי זמן שכיבה:
3,Berakhot,"(2, 4, 0)",הקטר חלבים ואברים,הקטר חלבים ואברים – של קרבנות שנזרק דמן ביום:
4,Berakhot,"(2, 4, 1)",מצותן,מצותן – להעלות כל הלילה ואינן נפסלים בלינה עד ...


In [8]:
df['Comment'][0]

'מאימתי קורין את שמע בערבין. משעה שהכהנים נכנסים לאכול בתרומתן – כהנים שנטמאו וטבלו והעריב שמשן והגיע עתם לאכול בתרומה:'

In [9]:
dedup= []
for i,rowi in df.iterrows():
    if(not dedup.__contains__(i)):
        for j,rowj in df.iterrows():
            if i != j and rowi['Starting Verse']==rowj['Starting Verse'] and not rowi['Starting Verse']=="":
                dedup.append(j)
                print(i,j,rowi['Starting Verse'],rowi['Location'],rowj['Location'])
                print(rowi['Comment'])
                print(rowj['Comment'])


2 9 עד שיעלה עמוד השחר  (2, 2, 0) (2, 4, 6)
עד שיעלה עמוד השחר – שכל הלילה קרוי זמן שכיבה:
עד שיעלה עמוד השחר – והוא מביאן להיות נותר דכתיב בתודה לא יניח ממנו עד בקר (ויקרא ז׳:ט״ו) וכלם מתודה ילמדו:
4 8 מצותן  (2, 4, 1) (2, 4, 5)
מצותן – להעלות כל הלילה ואינן נפסלים בלינה עד שיעלה עמוד השחר והן למטה מן המזבח דכתיב ולא ילין לבקר (שמות ל״ד:כ״ה):
מצותן – זמן אכילתן:
6 1224 אברים  (2, 4, 3) (51, 9, 0)
אברים – של עולה:
אברים – של עולות:
18 2705 אם כן  (3, 1, 0) (111, 5, 0)
אם כן – דהאי קרא וטהר לשון צווי הוא נימא קרא ויטהר מאי וטהר:
אם כן – דלית לך התם נכסי ואבוך לא שכיב התם:
87 253 מאי קרא  (6, 2, 0) (12, 14, 0)
מאי קרא – דכנור היה תלוי למעלה ממטתו ומעוררו
מאי קרא – דאף הוי כרגע דכתיב כי רגע באפו חיים ברצונו:
91 976 דאמר מר  (6, 5, 1) (43, 3, 0)
דאמר מר – במסכת דרך ארץ:
דאמר מר – גבי עשר תקנות בב"ק בפרק מרובה:
91 2174 דאמר מר  (6, 5, 1) (90, 10, 1)
דאמר מר – במסכת דרך ארץ:
דאמר מר – לעיל בפרק שני (ברכות דף טז.):
142 378 מאי משמע  (8, 3, 1) (18, 19, 0)
מאי משמע – דבקריאת שמע:
מאי משמע – דעל

KeyboardInterrupt: 

In [10]:
import torch
import os
import sys

from transformers import BertTokenizer, BertForMaskedLM
sys.path.append('./BEREL_2.0_PyTorch_Model_')
from rabtokenizer import RabbinicTokenizer
bert_path= "BEREL_2.0_PyTorch_Model_"
tokenizer = RabbinicTokenizer(BertTokenizer.from_pretrained(os.path.join(bert_path, 'vocab.txt')))
model = BertForMaskedLM.from_pretrained(bert_path)

/Users/ephraimmeiri/miniconda3/envs/Rashi Thesis work/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1756: FutureWarning: Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  warnings.warn(


In [11]:
print(model)

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(128000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

In [22]:
from sklearn.metrics.pairwise import cosine_similarity
text = "בראשית ברא אלהים את השמים ואת הארץ" 
inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
embed1 = model(**inputs)

In [26]:
type(embed1)

transformers.modeling_outputs.MaskedLMOutput

In [ ]:
text = "בראשונה ברא אלהים את הרקיע ואת הארץ"
inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
embed2= model(**inputs)
cosine_similarity(embed1.last_hidden_state.mean(dim=1).squeeze().detach().numpy(),embed2.last_hidden_state.mean(dim=1).squeeze().detach().numpy())

In [ ]:
def embed_text(text):
    """
    Convert text into embeddings using Berel model.
    """
    # Tokenize input text
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)

    # # Get embeddings
    # with torch.no_grad():
    #     outputs = model(**inputs)
    # embeddings = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

    # Pass text through model
    output = model.encode(**inputs)
    
    # Get predictions
    predictions = output.logits.argmax(-1)
    
    return embeddings

In [ ]:
# Convert each comment to embeddings
embedded_comments = [(embed_text(comment), daf_num, line_num,comment_num) for comment, daf_num, line_num, comment_num in comments_with_detailed_references]